# Install Modules

# Imports

In [10]:
# 1. Basic Python Imports
import pandas as pd
import numpy as np
import pickle as pi
import os

# 2. API PULL
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from defillama import DefiLlama

# 3. API Keys
from dotenv import load_dotenv


In [11]:
import coinmarketcapapi

cmc = coinmarketcapapi.CoinMarketCapAPI('3486d5cf-6e5b-443f-8f9a-140421f63bb3')

In [12]:
# Changing scientific notation of values in table 

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [13]:
# api pull
data_listing = cmc.cryptocurrency_listings_latest(convert='USD')

In [14]:
# create df and narrow columns
df = pd.DataFrame.from_dict(data_listing.data)

df = df[['name', 'symbol', 'date_added', 'quote']]
df

,name,symbol,date_added,quote
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,"{'USD': {'price': 36520.81614900267, 'volume_2..."
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,"{'USD': {'price': 2746.099053073823, 'volume_2..."
2,Tether,USDT,2015-02-25T00:00:00.000Z,"{'USD': {'price': 1.0000822215433238, 'volume_..."
3,BNB,BNB,2017-07-25T00:00:00.000Z,"{'USD': {'price': 378.0795404484726, 'volume_2..."
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,"{'USD': {'price': 1.0001125804285151, 'volume_..."
...,...,...,...,...
95,PAX Gold,PAXG,2019-09-26T00:00:00.000Z,"{'USD': {'price': 1879.3049614297095, 'volume_..."
96,0x,ZRX,2017-08-16T00:00:00.000Z,"{'USD': {'price': 0.7124045525815077, 'volume_..."
97,Moonbeam,GLMR,2022-01-11T12:55:50.000Z,"{'USD': {'price': 2.4353738493065054, 'volume_..."
98,ICON,ICX,2017-10-27T00:00:00.000Z,"{'USD': {'price': 0.6437903043985266, 'volume_..."


In [15]:
# isolate quotes column for expansion into columns
quotes_df = df[['name', 'quote']].copy()
df = df.drop(columns = ['quote'])

In [16]:
# expand into columns
usd = quotes_df["quote"].apply(pd.Series)
price = usd['USD'].apply(pd.Series)

In [17]:
# concat for final df
crypto_df = pd.concat([df, price], axis = 1, join = 'inner')

crypto_df.head(10)

,name,symbol,date_added,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36520.816,42883382989.613,17.159,0.313,-7.875,-8.223,-20.226,-4.820,-10.189,695054348848.275,41.493,766937139129.060,2022-05-05T23:33:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2746.099,22596747392.268,24.841,0.220,-6.402,-6.425,-20.041,7.401,-6.973,331346702220.367,19.781,331346702220.370,2022-05-05T23:32:00.000Z
2,Tether,USDT,2015-02-25T00:00:00.000Z,1.000,85431911870.871,19.073,0.002,0.006,-0.005,-0.011,-0.025,-0.031,83159214021.183,4.964,85720999348.840,2022-05-05T23:32:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,378.080,2381058124.989,38.901,0.169,-5.971,-7.063,-15.533,1.010,-4.569,61731683533.927,3.685,62427268741.140,2022-05-05T23:32:00.000Z
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,1.000,5124155571.561,15.527,-0.004,0.004,0.012,0.047,0.056,-0.014,48855106931.324,2.917,48855106931.320,2022-05-05T23:32:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.598,2183420545.070,12.451,0.426,-7.371,-7.367,-27.037,-17.659,-7.148,28910181579.098,1.726,59802083158.230,2022-05-05T23:32:00.000Z
6,Terra,LUNA,2019-07-26T00:00:00.000Z,82.580,2284349564.377,18.798,-0.421,-4.242,-6.982,-29.089,4.513,56.292,28350117843.744,1.692,59964736405.760,2022-05-05T23:32:00.000Z
7,Solana,SOL,2020-04-10T00:00:00.000Z,84.541,1809010898.986,28.729,0.474,-8.258,-13.799,-33.626,0.175,-23.337,28266250897.029,1.687,43252635128.200,2022-05-05T23:32:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.792,1802705149.763,20.207,0.452,-11.462,-6.397,-32.621,-3.879,-29.199,26730158592.070,1.596,35637502930.600,2022-05-05T23:32:00.000Z
9,TerraUSD,UST,2020-09-21T00:00:00.000Z,0.999,734564618.313,20.332,-0.230,-0.169,-0.076,0.019,-0.310,-0.080,18657774225.187,1.115,18657774225.190,2022-05-05T23:32:00.000Z


In [54]:
# Sorting values by largest market cap  

crypto_df.sort_values(by='market_cap', ascending=False).head(10)

,name,symbol,date_added,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36520.816,42883382989.613,17.159,0.313,-7.875,-8.223,-20.226,-4.820,-10.189,695054348848.275,41.493,766937139129.060,2022-05-05T23:33:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2746.099,22596747392.268,24.841,0.220,-6.402,-6.425,-20.041,7.401,-6.973,331346702220.367,19.781,331346702220.370,2022-05-05T23:32:00.000Z
2,Tether,USDT,2015-02-25T00:00:00.000Z,1.000,85431911870.871,19.073,0.002,0.006,-0.005,-0.011,-0.025,-0.031,83159214021.183,4.964,85720999348.840,2022-05-05T23:32:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,378.080,2381058124.989,38.901,0.169,-5.971,-7.063,-15.533,1.010,-4.569,61731683533.927,3.685,62427268741.140,2022-05-05T23:32:00.000Z
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,1.000,5124155571.561,15.527,-0.004,0.004,0.012,0.047,0.056,-0.014,48855106931.324,2.917,48855106931.320,2022-05-05T23:32:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.598,2183420545.070,12.451,0.426,-7.371,-7.367,-27.037,-17.659,-7.148,28910181579.098,1.726,59802083158.230,2022-05-05T23:32:00.000Z
6,Terra,LUNA,2019-07-26T00:00:00.000Z,82.580,2284349564.377,18.798,-0.421,-4.242,-6.982,-29.089,4.513,56.292,28350117843.744,1.692,59964736405.760,2022-05-05T23:32:00.000Z
7,Solana,SOL,2020-04-10T00:00:00.000Z,84.541,1809010898.986,28.729,0.474,-8.258,-13.799,-33.626,0.175,-23.337,28266250897.029,1.687,43252635128.200,2022-05-05T23:32:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.792,1802705149.763,20.207,0.452,-11.462,-6.397,-32.621,-3.879,-29.199,26730158592.070,1.596,35637502930.600,2022-05-05T23:32:00.000Z
9,TerraUSD,UST,2020-09-21T00:00:00.000Z,0.999,734564618.313,20.332,-0.230,-0.169,-0.076,0.019,-0.310,-0.080,18657774225.187,1.115,18657774225.190,2022-05-05T23:32:00.000Z


In [25]:
crypto_df['market_cap'].dtypes

dtype('float64')

In [30]:
crypto_df.loc['market_cap'] = crypto_df.loc[:, 'market_cap'].astype("float")

In [31]:
crypto_df.dtypes

name                         object
symbol                       object
date_added                   object
price                       float64
volume_24h                  float64
volume_change_24h           float64
percent_change_1h           float64
percent_change_24h          float64
percent_change_7d           float64
percent_change_30d          float64
percent_change_60d          float64
percent_change_90d          float64
market_cap                  float64
market_cap_dominance        float64
fully_diluted_market_cap    float64
last_updated                 object
dtype: object

In [34]:
# Eliminate the coins under $25B market cap 

crypto_df_sorted = crypto_df.loc[crypto_df['market_cap'] >= 25000000000].copy()
crypto_df_sorted.head(10)

,name,symbol,date_added,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36520.816,42883382989.613,17.159,0.313,-7.875,-8.223,-20.226,-4.820,-10.189,695054348848.275,41.493,766937139129.060,2022-05-05T23:33:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2746.099,22596747392.268,24.841,0.220,-6.402,-6.425,-20.041,7.401,-6.973,331346702220.367,19.781,331346702220.370,2022-05-05T23:32:00.000Z
2,Tether,USDT,2015-02-25T00:00:00.000Z,1.000,85431911870.871,19.073,0.002,0.006,-0.005,-0.011,-0.025,-0.031,83159214021.183,4.964,85720999348.840,2022-05-05T23:32:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,378.080,2381058124.989,38.901,0.169,-5.971,-7.063,-15.533,1.010,-4.569,61731683533.927,3.685,62427268741.140,2022-05-05T23:32:00.000Z
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,1.000,5124155571.561,15.527,-0.004,0.004,0.012,0.047,0.056,-0.014,48855106931.324,2.917,48855106931.320,2022-05-05T23:32:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.598,2183420545.070,12.451,0.426,-7.371,-7.367,-27.037,-17.659,-7.148,28910181579.098,1.726,59802083158.230,2022-05-05T23:32:00.000Z
6,Terra,LUNA,2019-07-26T00:00:00.000Z,82.580,2284349564.377,18.798,-0.421,-4.242,-6.982,-29.089,4.513,56.292,28350117843.744,1.692,59964736405.760,2022-05-05T23:32:00.000Z
7,Solana,SOL,2020-04-10T00:00:00.000Z,84.541,1809010898.986,28.729,0.474,-8.258,-13.799,-33.626,0.175,-23.337,28266250897.029,1.687,43252635128.200,2022-05-05T23:32:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.792,1802705149.763,20.207,0.452,-11.462,-6.397,-32.621,-3.879,-29.199,26730158592.070,1.596,35637502930.600,2022-05-05T23:32:00.000Z


In [40]:
# Out of the remaining coins, eliminate coins that are newer than 2020 

crypto_df_sorted_2 = crypto_df_sorted.loc[crypto_df_sorted['date_added'] <= '2020-01-01'].copy()
crypto_df_sorted_2.head(10)

,name,symbol,date_added,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36520.816,42883382989.613,17.159,0.313,-7.875,-8.223,-20.226,-4.820,-10.189,695054348848.275,41.493,766937139129.060,2022-05-05T23:33:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2746.099,22596747392.268,24.841,0.220,-6.402,-6.425,-20.041,7.401,-6.973,331346702220.367,19.781,331346702220.370,2022-05-05T23:32:00.000Z
2,Tether,USDT,2015-02-25T00:00:00.000Z,1.000,85431911870.871,19.073,0.002,0.006,-0.005,-0.011,-0.025,-0.031,83159214021.183,4.964,85720999348.840,2022-05-05T23:32:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,378.080,2381058124.989,38.901,0.169,-5.971,-7.063,-15.533,1.010,-4.569,61731683533.927,3.685,62427268741.140,2022-05-05T23:32:00.000Z
4,USD Coin,USDC,2018-10-08T00:00:00.000Z,1.000,5124155571.561,15.527,-0.004,0.004,0.012,0.047,0.056,-0.014,48855106931.324,2.917,48855106931.320,2022-05-05T23:32:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.598,2183420545.070,12.451,0.426,-7.371,-7.367,-27.037,-17.659,-7.148,28910181579.098,1.726,59802083158.230,2022-05-05T23:32:00.000Z
6,Terra,LUNA,2019-07-26T00:00:00.000Z,82.580,2284349564.377,18.798,-0.421,-4.242,-6.982,-29.089,4.513,56.292,28350117843.744,1.692,59964736405.760,2022-05-05T23:32:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.792,1802705149.763,20.207,0.452,-11.462,-6.397,-32.621,-3.879,-29.199,26730158592.070,1.596,35637502930.600,2022-05-05T23:32:00.000Z


In [55]:
# Eliminate Stablecoins - Struggling syntax to retain Terra Luna in this dataframe @Lucas

crypto_df_sorted_3 = crypto_df_sorted_2.loc[crypto_df_sorted_2['percent_change_90d'] < -1].copy()
crypto_df_sorted_3.head(10)

,name,symbol,date_added,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36520.816,42883382989.613,17.159,0.313,-7.875,-8.223,-20.226,-4.820,-10.189,695054348848.275,41.493,766937139129.060,2022-05-05T23:33:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2746.099,22596747392.268,24.841,0.220,-6.402,-6.425,-20.041,7.401,-6.973,331346702220.367,19.781,331346702220.370,2022-05-05T23:32:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,378.080,2381058124.989,38.901,0.169,-5.971,-7.063,-15.533,1.010,-4.569,61731683533.927,3.685,62427268741.140,2022-05-05T23:32:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.598,2183420545.070,12.451,0.426,-7.371,-7.367,-27.037,-17.659,-7.148,28910181579.098,1.726,59802083158.230,2022-05-05T23:32:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.792,1802705149.763,20.207,0.452,-11.462,-6.397,-32.621,-3.879,-29.199,26730158592.070,1.596,35637502930.600,2022-05-05T23:32:00.000Z


In [61]:
#Testing dropping 2 colums

crypto_df_sorted_4 = crypto_df_sorted_3.drop(['volume_24h', 'volume_change_24h'], axis=1)
crypto_df_sorted_4

,name,symbol,date_added,price,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,Bitcoin,BTC,2013-04-28T00:00:00.000Z,36520.816,0.313,-7.875,-8.223,-20.226,-4.820,-10.189,695054348848.275,41.493,766937139129.060,2022-05-05T23:33:00.000Z
1,Ethereum,ETH,2015-08-07T00:00:00.000Z,2746.099,0.220,-6.402,-6.425,-20.041,7.401,-6.973,331346702220.367,19.781,331346702220.370,2022-05-05T23:32:00.000Z
3,BNB,BNB,2017-07-25T00:00:00.000Z,378.080,0.169,-5.971,-7.063,-15.533,1.010,-4.569,61731683533.927,3.685,62427268741.140,2022-05-05T23:32:00.000Z
5,XRP,XRP,2013-08-04T00:00:00.000Z,0.598,0.426,-7.371,-7.367,-27.037,-17.659,-7.148,28910181579.098,1.726,59802083158.230,2022-05-05T23:32:00.000Z
8,Cardano,ADA,2017-10-01T00:00:00.000Z,0.792,0.452,-11.462,-6.397,-32.621,-3.879,-29.199,26730158592.070,1.596,35637502930.600,2022-05-05T23:32:00.000Z


Next we are finding our DeFi set of coinns

In [ ]:
# initialize api client|
llama = DefiLlama()

# Get all protocols data
response = llama.get_all_protocols()

# Get a protocol data
response_1 = llama.get_protocol(name='uniswap')

# Get historical values of total TVL
response_2 = llama.get_historical_tvl()

# Get protocol TVL
response_3 = llama.get_protocol_tvl(name='uniswap')

In [ ]:
# Created a DataFrame to structure by relevant parameters

defi_df = pd.DataFrame(response, 
                    columns =[
                        'name',
                        'symbol',
                        'tvl',
                        'fdv',
                        'mcap'
                    ])

defi_df.set_index('name',inplace=True)
defi_df.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
defi_df1 = pd.DataFrame.from_dict(response)

defi_df1 = defi_df1[['name','symbol', 'tvl', 'fdv', 'mcap']]
defi_df1.set_index('symbol', inplace=True)
defi_df1


In [ ]:
# Sort by highest TVL's 

defi_df.sort_values(by='tvl', ascending=False).head()

In [ ]:
defi_df.dtypes

In [ ]:
defi_df1= defi_df.loc[defi_df['tvl'] > 9000000000].copy()
defi_df1.sort_values(by='mcap', ascending=False).head(10)

In [ ]:
defi_df2 = defi_df1.loc[defi_df1['mcap'] < 10000000000].copy()
defi_df2.sort_values(by='tvl', ascending=False).head(10)

In [ ]:
defi_df2['mcap_tvl_ratio'] = defi_df2['mcap'] / defi_df2['tvl'] 
defi_df2.sort_values(by='mcap_tvl_ratio', ).head(10)

In [ ]:
defi_df3 = defi_df2.loc[defi_df2['mcap_tvl_ratio'] < 0.5].copy()
defi_df3.sort_values(by='tvl', ascending=False).head(10)

In [ ]:
defi_df3['fdv_tvl_ratio'] = defi_df3['fdv'] / defi_df3['tvl']
defi_df3

In [ ]:
# Market Cap / Total Value Locked RATIO :

# From a theoretical standpoint, the higher the TVL ratio is, the lower the value of an asset needs to be; however, this is not always the case when 
# we look at reality. One of the easiest ways to implement the TVL ratio is to help determine if a DeFi asset is undervalued or overvalued, and this can be done by looking at the ratio.
# If it is under 1/closer to 0, it is undervalued in most cases. 


In [ ]:
# Fully Dilluted Value / Total Value Locked RATIO: 

# Similar to MCAP/TVL , FDV/TVL helps us tell the whole story and is also measured the same. The closer the ratio is to zero, typically the coin is viewed as undervalued. 
#If it is over 1, it may be overvalued. Using both metrics helps provide us with the full picture.  

In [ ]:
# Understanding Fully Diluted Valuation (FDV)

#The Fully Diluted Valuation of a cryptocurrency or token is what the digital asset's market cap would be if all the coins or tokens in its total supply were issued. It’s calculated by
#multiplying the current market price of a particular coin or token with the maximum number of coins there will be.

# Now we need to import historical prices for the coins and find price relationship and correlation (variance, cov etc) 